In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget  https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz

--2022-03-03 09:44:58--  https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272637746 (260M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop2.7.tgz’

spark-3.2.1-bin-had 100%[===================>] 260.01M   226MB/s    in 1.2s    

2022-03-03 09:45:10 (226 MB/s) - ‘spark-3.2.1-bin-hadoop2.7.tgz’ saved [272637746/272637746]



In [ ]:
!tar xf spark-3.2.1-bin-hadoop2.7.tgz


In [ ]:
!pip install -q findspark

In [ ]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
from pyspark.sql.functions import udf, lit, when, date_sub, col
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField, StringType
from pyspark.sql.types import BooleanType, DateType, DoubleType, FloatType
import json
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import Row
from datetime import datetime
from pyspark.sql.functions import current_date

# conf = SparkConf().setAppName('Colab').setMaster('local')
# sc = SparkContext(conf=spark)
sqlContext = SQLContext(spark)

/content/spark-3.2.1-bin-hadoop2.7/python/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


### Stock data sample

In [ ]:
upload_path = "/content/data"

checkpoint_path = '/content/stocks_data/_checkpoints'
write_path = '/content/stocks_data'

In [ ]:
df2 = spark.read.option("header",True).csv(upload_path)
cust_schema = df2.schema

In [ ]:
# Set up the stream to begin reading incoming files from the
# upload_path location.
df = spark.readStream.format('cloudFiles').option('cloudFiles.format', 'csv').option('header', 'true').schema(cust_schema).load(upload_path)

# Start the stream.
# Use the checkpoint_path location to keep a record of all files that
# have already been uploaded to the upload_path location.
# For those that have been uploaded since the last check,
# write the newly-uploaded files' data to the write_path location.
df.writeStream.format('parquet') \
  .option('checkpointLocation', checkpoint_path) \
  .option('path', write_path) \
  .start()

Py4JJavaError: ignored

In [ ]:
#  Reading all the csv files from stocks dir in colab
from  pyspark.sql.functions import input_file_name
df = spark.read.option("header",True).csv('/content/stocks/*').withColumn("filename", input_file_name())

In [ ]:
df.head(5)

[Row(SC_CODE='500002', SC_NAME='ABB LTD.    ', SC_GROUP='A ', SC_TYPE='Q', OPEN='1860.25', HIGH='1892.00', LOW='1846.00', CLOSE='1873.20', LAST='1864.00', PREVCLOSE='1854.95', NO_TRADES='768', NO_OF_SHRS='4120', NET_TURNOV='7666507.00', TDCLOINDI=None, filename='file:/content/stocks/EQ041021.CSV'),
 Row(SC_CODE='500003', SC_NAME='AEGIS LOGIS ', SC_GROUP='A ', SC_TYPE='Q', OPEN='230.25', HIGH='233.35', LOW='228.00', CLOSE='229.35', LAST='229.35', PREVCLOSE='228.95', NO_TRADES='2475', NO_OF_SHRS='54056', NET_TURNOV='12441733.00', TDCLOINDI=None, filename='file:/content/stocks/EQ041021.CSV'),
 Row(SC_CODE='500008', SC_NAME='AMAR RAJA BA', SC_GROUP='A ', SC_TYPE='Q', OPEN='774.50', HIGH='774.85', LOW='762.00', CLOSE='766.15', LAST='766.15', PREVCLOSE='765.50', NO_TRADES='2642', NO_OF_SHRS='63238', NET_TURNOV='48593068.00', TDCLOINDI=None, filename='file:/content/stocks/EQ041021.CSV'),
 Row(SC_CODE='500009', SC_NAME='A.SARABHAI  ', SC_GROUP='X ', SC_TYPE='Q', OPEN='30.75', HIGH='31.00', LOW

In [ ]:
import os 

def get_date(value):
  f_name, _ = os.path.splitext(os.path.basename(value))
  col_date = datetime. strptime(f_name[-6:], '%d%m%y').date()
  return str(col_date)

print(get_date('file:/content/stocks/EQ041021.CSV'))

get_date_udf_func = udf(get_date,StringType()) 
# spark.udf.register("get_date_udf_func", get_date,StringType())

2021-10-04


In [ ]:
from pyspark.sql.functions import month , year, dayofmonth

df = df.withColumn("closing_date", get_date_udf_func(df["filename"]))

In [ ]:
df.show(1, truncate=False)
df_path = 'file:/content/stock_merged.csv'

+-------+------------+--------+-------+-------+-------+-------+-------+-------+---------+---------+----------+----------+---------+---------------------------------+------------+
|SC_CODE|SC_NAME     |SC_GROUP|SC_TYPE|OPEN   |HIGH   |LOW    |CLOSE  |LAST   |PREVCLOSE|NO_TRADES|NO_OF_SHRS|NET_TURNOV|TDCLOINDI|filename                         |closing_date|
+-------+------------+--------+-------+-------+-------+-------+-------+-------+---------+---------+----------+----------+---------+---------------------------------+------------+
|500002 |ABB LTD.    |A       |Q      |1860.25|1892.00|1846.00|1873.20|1864.00|1854.95  |768      |4120      |7666507.00|null     |file:/content/stocks/EQ041021.CSV|2021-10-04  |
+-------+------------+--------+-------+-------+-------+-------+-------+-------+---------+---------+----------+----------+---------+---------------------------------+------------+
only showing top 1 row



In [ ]:
groups = [x[0] for x in df.select("SC_CODE").distinct().collect()]
groups[:5]

'508905'

In [ ]:
groups[:5]

['500148', '500253', '501314', '507514', '507525']

In [ ]:
df.write.options(delimiter='\u0001').option('nullValue', None).mode("overwrite").csv(df_path)

In [ ]:
new_pd = df.toPandas()
new_pd.head()

,SC_CODE,SC_NAME,SC_GROUP,SC_TYPE,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,NO_TRADES,NO_OF_SHRS,NET_TURNOV,TDCLOINDI,filename,closing_date
0,500002,ABB LTD.,A,Q,1860.25,1892.00,1846.00,1873.20,1864.00,1854.95,768,4120,7666507.00,None,file:/content/stocks/EQ041021.CSV,2021-10-04
1,500003,AEGIS LOGIS,A,Q,230.25,233.35,228.00,229.35,229.35,228.95,2475,54056,12441733.00,None,file:/content/stocks/EQ041021.CSV,2021-10-04
2,500008,AMAR RAJA BA,A,Q,774.50,774.85,762.00,766.15,766.15,765.50,2642,63238,48593068.00,None,file:/content/stocks/EQ041021.CSV,2021-10-04
3,500009,A.SARABHAI,X,Q,30.75,31.00,30.55,30.65,30.65,30.65,728,145196,4462266.00,None,file:/content/stocks/EQ041021.CSV,2021-10-04
4,500010,HDFC,A,Q,2740.00,2769.00,2721.05,2725.00,2725.00,2712.25,6175,83389,228666721.00,None,file:/content/stocks/EQ041021.CSV,2021-10-04


In [ ]:
new_pd.shape

(744838, 16)

In [ ]:
new_pd.to_csv('stock_merge.csv')

In [ ]:
df_merged = spark.read.option("header",True).csv('file:/content/stock_merge.csv')
df_merged.show()

+---+-------+------------+--------+-------+-------+--------+-------+--------+--------+---------+---------+----------+------------+---------+--------------------+------------+
|_c0|SC_CODE|     SC_NAME|SC_GROUP|SC_TYPE|   OPEN|    HIGH|    LOW|   CLOSE|    LAST|PREVCLOSE|NO_TRADES|NO_OF_SHRS|  NET_TURNOV|TDCLOINDI|            filename|closing_date|
+---+-------+------------+--------+-------+-------+--------+-------+--------+--------+---------+---------+----------+------------+---------+--------------------+------------+
|  0| 500002|ABB LTD.    |      A |      Q|1860.25| 1892.00|1846.00| 1873.20| 1864.00|  1854.95|      768|      4120|  7666507.00|     null|file:/content/sto...|  2021-10-04|
|  1| 500003|AEGIS LOGIS |      A |      Q| 230.25|  233.35| 228.00|  229.35|  229.35|   228.95|     2475|     54056| 12441733.00|     null|file:/content/sto...|  2021-10-04|
|  2| 500008|AMAR RAJA BA|      A |      Q| 774.50|  774.85| 762.00|  766.15|  766.15|   765.50|     2642|     63238| 4859306

### Agg

In [ ]:
data1  = [{'Name':'Jhon','ID':21.528,'Add':'USA'},{'Name':'Joe','ID':3.69,'Add':'USA'},{'Name':'Tina','ID':2.48,'Add':'IND'},{'Name':'Jhon','ID':22.22, 'Add':'USA'},{'Name':'Joe','ID':5.33,'Add':'INA'}]

In [ ]:
test_df = sqlContext.createDataFrame(spark.sparkContext.parallelize(data1))
test_df.show()

+---+------+----+
|Add|    ID|Name|
+---+------+----+
|USA|21.528|Jhon|
|USA|  3.69| Joe|
|IND|  2.48|Tina|
|USA| 22.22|Jhon|
|INA|  5.33| Joe|
+---+------+----+



In [ ]:
# count
c = test_df.groupBy('Name')
c.count().show()

+----+-----+
|Name|count|
+----+-----+
| Joe|    2|
|Jhon|    2|
|Tina|    1|
+----+-----+



In [ ]:
# agg functions
# c.agg({'ID':'sum'}).show()
# c.agg({'ID':'count'}).show()
# c.agg({'ID':'avg'}).show()
# c.agg({'ID':'max'}).show()
# c.agg({'ID':'last'}).show()
c.agg({'ID':'collect_list'}).show()

+----+----------------+
|Name|collect_list(ID)|
+----+----------------+
| Joe|    [3.69, 5.33]|
|Jhon| [21.528, 22.22]|
|Tina|          [2.48]|
+----+----------------+



In [ ]:
def plus_mean(pandas_df):
    pandas_df = pandas_df.assign(test=pandas_df.ID.median())
    pandas_df = pandas_df.assign(test2=pandas_df.ID.max())
    return pandas_df

test_df = test_df.withColumn('test', lit(None).cast(FloatType()))
test_df = test_df.withColumn('test2', lit(None).cast(FloatType()))
test_df.groupby('Name').applyInPandas(plus_mean, schema=test_df.schema).show()

+---+------+----+------+-----+
|Add|    ID|Name|  test|test2|
+---+------+----+------+-----+
|USA|21.528|Jhon|21.874|22.22|
|USA| 22.22|Jhon|21.874|22.22|
|USA|  3.69| Joe|  4.51| 5.33|
|INA|  5.33| Joe|  4.51| 5.33|
|IND|  2.48|Tina|  2.48| 2.48|
+---+------+----+------+-----+



In [ ]:
test_df.select("name").distinct().collect()

[Row(name='Joe'), Row(name='Jhon'), Row(name='Tina')]

In [ ]:
# select , withColumn
# test_df.select(test_df.columns[0:3]).show()

test_df = test_df.withColumn("ID",col("ID").cast("Integer"))
test_df.printSchema()

test_df = test_df.withColumnRenamed("Add","Address")
# Adding MULTIPLE columns.
test_df.withColumn("New_Column",lit("NEW")).withColumn("New_Column2",col("Address")).show()

root
 |-- Add: string (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- test: float (nullable = true)

+-------+---+----+----+----------+-----------+
|Address| ID|Name|test|New_Column|New_Column2|
+-------+---+----+----+----------+-----------+
|    USA| 21|Jhon|null|       NEW|        USA|
|    USA|  3| Joe|null|       NEW|        USA|
|    IND|  2|Tina|null|       NEW|        IND|
|    USA| 22|Jhon|null|       NEW|        USA|
|    INA|  5| Joe|null|       NEW|        INA|
+-------+---+----+----+----------+-----------+



### join 

In [ ]:
data1  = [{'Name':'Jhon','ID':21.528,'Add':'USA'},{'Name':'Joe','ID':3.69,'Add':'USA'},{'Name':'Tina','ID':2.48,'Add':'IND'},{'Name':'Jhon','ID':22.22, 'Add':'USA'},{'Name':'Joe','ID':5.33,'Add':'INA'}]

data2  = [{'Name':'Jhon','ID':21.528,'Add':'USA'},{'Name':'Joe','ID':3.69,'Add':'USA'},{'Name':'Tinya','ID':2.44,'Add':'IND'},{'Name':'Jhond','ID':22.2, 'Add':'USA'},{'Name':'Joe','ID':5,'Add':'IND'}]


In [ ]:
test_df1 = sqlContext.createDataFrame(spark.sparkContext.parallelize(data1))
test_df2 = sqlContext.createDataFrame(spark.sparkContext.parallelize(data2))

NameError: ignored

In [ ]:
df_inner = test_df1.join(test_df2 , on=['Name']  , how = 'inner')
df_inner.show()

In [ ]:
test_df2.show(1, vertical=True)

In [ ]:
test_df2.take(1)

### Ranking Function

In [ ]:
# ROW_NUMBER()
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

data1 = (("Bob", "IT", 4500), \
("Maria", "IT", 4600), \
("James", "IT", 3850), \
("Maria", "HR", 4500),  ("Maria2", "HR", 5200),\
("James", "IT", 4500), \
("Sam", "HR", 3300), \
("Jen", "HR", 4450), ("Jen2", "HR", 4800),\
("Jeff", "Marketing", 4500), \
("Anand", "Marketing", 2000),\
("Shaid", "IT", 3850) \
)
col= ["Name", "MBA_Stream", "SEM_MARKS"]
b = spark.createDataFrame(data1,col)
w = Window.partitionBy("MBA_Stream").orderBy("SEM_MARKS")

b.withColumn("Windowfunc_row",row_number().over(w)).show()

ModuleNotFoundError: ignored

In [ ]:
# RANK Function
from pyspark.sql.functions import rank


b.withColumn("Window_Rank",rank().over(w)).show()

+------+----------+---------+-----------+
|  Name|MBA_Stream|SEM_MARKS|Window_Rank|
+------+----------+---------+-----------+
|   Sam|        HR|     3300|          1|
|   Jen|        HR|     4450|          2|
| Maria|        HR|     4500|          3|
|  Jen2|        HR|     4800|          4|
|Maria2|        HR|     5200|          5|
| James|        IT|     3850|          1|
| Shaid|        IT|     3850|          1|
|   Bob|        IT|     4500|          3|
| James|        IT|     4500|          3|
| Maria|        IT|     4600|          5|
| Anand| Marketing|     2000|          1|
|  Jeff| Marketing|     4500|          2|
+------+----------+---------+-----------+



In [ ]:
# ntile
from pyspark.sql.functions import ntile
b.withColumn("Window_Ntile",ntile(2).over(w)).show()

+------+----------+---------+------------+
|  Name|MBA_Stream|SEM_MARKS|Window_Ntile|
+------+----------+---------+------------+
|   Sam|        HR|     3300|           1|
|   Jen|        HR|     4450|           1|
| Maria|        HR|     4500|           1|
|  Jen2|        HR|     4800|           2|
|Maria2|        HR|     5200|           2|
| James|        IT|     3850|           1|
| Shaid|        IT|     3850|           1|
|   Bob|        IT|     4500|           1|
| James|        IT|     4500|           2|
| Maria|        IT|     4600|           2|
| Anand| Marketing|     2000|           1|
|  Jeff| Marketing|     4500|           2|
+------+----------+---------+------------+



### SubString 

In [ ]:
from pyspark.sql.functions import lit, concat

a= spark.createDataFrame(["SAM","JOHN","AND","ROBIN","ANAND"], "string").toDF("Name")
# b=a.withColumn("Sub_Name",a.Name.substr(1,5)).show()
b = a.withColumn("Concated_Value", concat(a.Name.substr(-5,5),lit("--"),a.Name.substr(1,2))).show()

+-----+--------------+
| Name|Concated_Value|
+-----+--------------+
|  SAM|       SAM--SA|
| JOHN|      JOHN--JO|
|  AND|       AND--AN|
|ROBIN|     ROBIN--RO|
|ANAND|     ANAND--AN|
+-----+--------------+



### PySpark read parquet

In [ ]:
import os
dir_path = os.path.dirname(os.path.realpath('spark_detail.ipynb'))
dir_path

'/content'

In [ ]:
data1 = (("Bob", "IT", 4500), \
("Maria", "IT", 4600),  \
("James", "IT", 3850),   \
("Maria", "HR", 4500),  \
("James", "IT", 4500),    \
("Sam", "HR", 3300),  \
("Jen", "HR", 3900),    \
("Jeff", "Marketing", 4500), \
("Anand", "Marketing", 2000),\
("Shaid", "IT", 3850) \
)
col= ["Name", "MBA_Stream", "SEM_MARKS"]
b = spark.createDataFrame(data1,col)
b.printSchema()

b.write.parquet(dir_path + "/test")

root
 |-- Name: string (nullable = true)
 |-- MBA_Stream: string (nullable = true)
 |-- SEM_MARKS: long (nullable = true)



In [ ]:
spark.read.parquet(dir_path + "/test").show(5)

+-----+----------+---------+
| Name|MBA_Stream|SEM_MARKS|
+-----+----------+---------+
|  Bob|        IT|     4500|
|Maria|        IT|     4600|
|James|        IT|     3850|
|Maria|        HR|     4500|
|James|        IT|     4500|
+-----+----------+---------+
only showing top 5 rows



### PYSPARK COLUMN TO LIST

In [ ]:
data1 = [{'Name':'Jhon','ID':21.528,'Add':'USA'},{'Name':'Joe','ID':3.69,'Add':'USA'},{'Name':'Tina','ID':2.48,'Add':'IND'},{'Name':'Jhon','ID':22.22, 'Add':'USA'},{'Name':'Joe','ID':5.33,'Add':'INA'}]

a = spark.sparkContext.parallelize(data1)
b = spark.createDataFrame(a)


In [ ]:
# using lambda

b.rdd.map(lambda x: x[2]).collect()

['Jhon', 'Joe', 'Tina', 'Jhon', 'Joe']

In [ ]:
# using flatMap

b.select(b.Name).rdd.flatMap(lambda x: x).collect()

['Jhon', 'Joe', 'Tina', 'Jhon', 'Joe']

### Structtype

In [ ]:
nes_Sch = StructType(
    [
        StructField(
            "Name",
            StructType(
                [
                    StructField("f_name", StringType(), True),
                    StructField("l_name", StringType(), True),
                ]
            ),
        ),
        StructField("ID", StringType(), True),
        StructField("Add", StringType(), True),
    ]
)
data1 = [(("John","cena"),"123","UK"),(("Singh","dd"),"234","IND")] 
b = spark.createDataFrame(data1,nes_Sch)
b.show()
b.printSchema()

+------------+---+---+
|        Name| ID|Add|
+------------+---+---+
|{John, cena}|123| UK|
| {Singh, dd}|234|IND|
+------------+---+---+

root
 |-- Name: struct (nullable = true)
 |    |-- f_name: string (nullable = true)
 |    |-- l_name: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Add: string (nullable = true)



In [ ]:
b[['Name.f_name']].show()

+------+
|f_name|
+------+
|  John|
| Singh|
+------+



### PYSPARK EXPLODE

In [ ]:
from pyspark.sql.functions import explode

data1  = [("Jhon",[["USA","MX","USW","UK"],["23","34","56"]]),("Joe",[["IND","AF","YR","QW"],["22","35","76"]]),("Juhi",[["USA","MX","USW","UK"],["13","64","59"]]),("Jhony",[["USSR","MXR","USA","UK"],["22","44","76"]])]


data_frame = spark.createDataFrame(data=data1, schema = ['name','subjectandID'])

In [ ]:
data_frame.printSchema()
data_frame.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- subjectandID: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)

+-----+------------------------------------+
|name |subjectandID                        |
+-----+------------------------------------+
|Jhon |[[USA, MX, USW, UK], [23, 34, 56]]  |
|Joe  |[[IND, AF, YR, QW], [22, 35, 76]]   |
|Juhi |[[USA, MX, USW, UK], [13, 64, 59]]  |
|Jhony|[[USSR, MXR, USA, UK], [22, 44, 76]]|
+-----+------------------------------------+



In [ ]:
df2 = data_frame.select(data_frame.name,explode(data_frame.subjectandID))
df2.printSchema()
df2.show()

root
 |-- name: string (nullable = true)
 |-- col: array (nullable = true)
 |    |-- element: string (containsNull = true)

+-----+--------------------+
| name|                 col|
+-----+--------------------+
| Jhon|  [USA, MX, USW, UK]|
| Jhon|        [23, 34, 56]|
|  Joe|   [IND, AF, YR, QW]|
|  Joe|        [22, 35, 76]|
| Juhi|  [USA, MX, USW, UK]|
| Juhi|        [13, 64, 59]|
|Jhony|[USSR, MXR, USA, UK]|
|Jhony|        [22, 44, 76]|
+-----+--------------------+



In [ ]:
import os 
from datetime import datetime
import re

def get_date(value):
  f_name, _ = os.path.splitext(os.path.basename(value))
  col_date = datetime. strptime(f_name[-6:], '%d%m%y').date()
  return str(col_date)

print(get_date('file:/content/stocks/EQ041021.CSV'))

2021-10-04
